In [166]:
import tensorflow as tf
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join
%matplotlib inline

In [167]:
def get_filename():
    mypath = './data/data1/'
    data_lidar = sorted(glob.glob(mypath + '*txt'), key=os.path.basename)
    data_leftcamera = sorted(glob.glob(mypath + 'left*'), key=os.path.basename)
    data_leftcamera = data_leftcamera[:len(data_lidar)]
    data_rightcamera = sorted(glob.glob(mypath + 'right*'), key=os.path.basename)
    data_rightcamera = data_rightcamera[:752]
    print "len lidar: ", len(data_lidar)
    print "len left: ", len(data_leftcamera)
    print "len right: ", len(data_rightcamera)
    return [data_lidar, data_leftcamera, data_rightcamera]


In [168]:
# only get serial number
def get_filenumber():
    mypath = './data/data1/'
    data_lidar = sorted(glob.glob(mypath + '*txt'), key=os.path.basename)
    file_numbers = []
    for i in range(len(data_lidar)):
        number = data_lidar[i]
        number = number.replace("./data/data1/","")
        number = number.replace(".txt","")
        file_numbers.append(number)
    print "number of sample: ", len(file_numbers)
    return file_numbers


In [169]:
data_lidar, data_leftcamera, data_rightcamera = read_data()
data_lidar

len lidar:  752
len left:  752
len right:  752


['./data/data1/5911492155.txt',
 './data/data1/5911492156.txt',
 './data/data1/5911492157.txt',
 './data/data1/5911492158.txt',
 './data/data1/5911492159.txt',
 './data/data1/5911492160.txt',
 './data/data1/5911492161.txt',
 './data/data1/5911492162.txt',
 './data/data1/5911492163.txt',
 './data/data1/5911492164.txt',
 './data/data1/5911492165.txt',
 './data/data1/5911492166.txt',
 './data/data1/5911492167.txt',
 './data/data1/5911492168.txt',
 './data/data1/5911492169.txt',
 './data/data1/5911492170.txt',
 './data/data1/5911492171.txt',
 './data/data1/5911492172.txt',
 './data/data1/5911492173.txt',
 './data/data1/5911492174.txt',
 './data/data1/5911492175.txt',
 './data/data1/5911492176.txt',
 './data/data1/5911492177.txt',
 './data/data1/5911492178.txt',
 './data/data1/5911492179.txt',
 './data/data1/5911492180.txt',
 './data/data1/5911492181.txt',
 './data/data1/5911492182.txt',
 './data/data1/5911492183.txt',
 './data/data1/5911492184.txt',
 './data/data1/5911492185.txt',
 './data

In [171]:
file_numbers = get_filenumber()

number of sample:  752


In [224]:
def read_my_file_format(file_numbers_queue):
    # reconstruce filename for three source
    left_image_filename_queue = []
    left_image_filename_queue = []
    lidar_filename_queue = []
    for i in range(len(file_numbers_queue)):
        left_image_filename = "./data/data1/left" + file_numbers_queue[i] + ".png"
        right_image_filename = "./data/data1/left" + file_numbers_queue[i] + ".png"
        lidar_filename = "./data/data1/" + file_numbers_queue[i] + ".txt"
        left_image_filename_queue.append(left_image_filename)
        right_image_filename_queue.append(right_image_filename)
        lidar_filename_queue.append(lidar_filename)
        
    # read and decode image    
    reader = tf.WholeFileReader()
    key_left, left_value = reader.read(left_image_filename_queue)
    key_right, right_value = reader.read(right_image_filename_queue)
    left_image = tf.image.decode_png(left_value)
    right_image = tf.image.decode_png(right_value)

    # read and decode lidar    
    lidar_reader = tf.TextLineReader()
    lidar_key, lidar_value = lidar_reader.read(lidar_filename_queue)
    record_defaults = [[] for col in range(120)]
    lidar_samples = tf.decode_csv(lidar_value, record_defaults=record_defaults)
    lidar_label = tf.pack(lidar_samples)

    return left_image, right_image, lidar_label

In [156]:
def input_pipeline(example_filenames,label_filenames, batch_size, read_threads, num_epochs=None):
    example_filename_queue = tf.train.string_input_producer(example_filenames, num_epochs=num_epochs, shuffle=False)
    label_filenames_queue = tf.train.string_input_producer(label_filenames, num_epochs=num_epochs, shuffle=False)
    example_list = [read_my_file_format(example_filename_queue, label_filenames_queue)for _ in range(read_threads)]
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch_join(
      example_list, batch_size=batch_size, capacity=capacity,
      min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch


In [157]:
def lidar_reader(filename):
    lidar_np = np.loadtxt(filename)
    lidar_tensor = tf.constant(lidar_np)
    return lidar_tensor
    

In [158]:
def lidar_decoder(lidar_string):
    lidar_np = np.fromstring(lidar_string)
    lidar_tensor = tf.constant(lidar_np)
    return lidar_tensor

In [172]:
leftcamera_filename_queue = tf.train.string_input_producer(data_leftcamera)
rightcamera_filename_queue = tf.train.string_input_producer(data_rightcamera)
lidar_filename_queue = tf.train.string_input_producer(data_lidar)

In [161]:
# left camera reader
leftcamera_reader = tf.WholeFileReader()
leftcamera_file_name, leftcamera_file = leftcamera_reader.read(leftcamera_filename_queue)
left_image = tf.image.decode_png(leftcamera_file)

In [219]:
# lidar reader
# lidar_filename_queue = tf.train.string_input_producer(['a.txt','a.txt'])
lidar_reader = tf.TextLineReader()
lidar_key, lidar_value = lidar_reader.read(lidar_filename_queue)
record_defaults = [[] for col in range(120)]
lidar_samples = tf.decode_csv(lidar_value, record_defaults=record_defaults, field_delim=',')
lidar_label = tf.pack(lidar_samples)

In [223]:
# Start a new session to show example output.
with tf.Session() as sess:
    # Required to get the filename matching to run.
    tf.initialize_all_variables().run()

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    

#     # Get an image tensor and print its value.
#     image_tensor = sess.run(left_image) # read single image
#     print image_tensor.shape
#     plt.imshow(image_tensor)

    # Get lidar info
    lidar_value = sess.run(lidar_label)
#     lidar_np = np.fromstring(lidar_string, sep='\n')
    print(lidar_label.eval())
    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)

[ 4.65924978  4.65525007  4.59600019  4.59399986  4.53725004  4.48024988
  4.5317502   4.47574997  4.47574997  4.47650003  4.46850014  4.46850014
  4.46750021  4.41575003  4.46400023  4.41575003  7.95825005  7.95825005
  7.95825005  7.95825005  7.95825005  7.95825005  7.95825005  7.95825005
  7.95825005  7.95825005  7.95825005  7.95825005  7.95825005  7.95825005
  7.95825005  7.95825005         inf         inf  4.65224981  4.71225023
  4.71425009  4.71624994  4.71850014  4.78350019  1.86500001  1.78900003
  1.73150003  1.66900003  1.67674994  1.68575001  1.70725     1.722
  1.73300004  1.74000001  1.755       1.78649998  1.79174995  1.81575
  1.83000004  1.85249996  1.87349999  1.89824998  1.85099995  1.801
  1.77049994  1.74325001  1.67750001  1.64499998  1.62        1.59449995
  1.57124996  1.52100003  1.49925005  1.47800004  1.45325005  1.43725002
  1.40100002  1.37975001  1.36749995  1.34949994  1.32175004  1.3075
  1.29674995  1.27950001  1.25450003  1.24399996  1.22774994  1.2217